In [ ]:
pip install pandasql


  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=0c5b5bd92916b1969886ce62f6ced11d087669a3345e863dace3561794c42980
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [ ]:
import pandasql as ps
import statsmodels.api as sm
import pandas as pd
import os

In [20]:
# Set the file path for this exercise
# basepath=r"D:\Teaching\Duke\2024\Data\Python"
# Import CRSP
basepath=r"/content/msf0820_short (1).dta"
rawmsf=pd.read_stata(os.path.join(basepath, r"/content/msf0820_short (1).dta"))
# Some variables may contain upper-case letters.
rawmsf.columns=map(str.lower, rawmsf.columns)
rawmsf=rawmsf[['permno', 'ret', 'prc', 'date', 'vwretd']]
# Clean some potential duplicate observations
rawmsf=rawmsf.drop_duplicates(subset=['permno', 'date'])

In [21]:
# Align day windows
def MergeDateWindow(dset, datevar, interval):
  dset['begdate']=dset[datevar]-pd.Timedelta(days=interval)
  dset['enddate']=dset[datevar]+pd.Timedelta(days=interval)
  return dset

rawmsf=MergeDateWindow(rawmsf, 'date', 5)

In [23]:
rawff=pd.read_stata(os.path.join(basepath, r"/content/ff0820.dta"))

crsp_ff_join='''select distinct a.*, b.mktrf, b.smb, b.hml, b.rf, b.dateff
  from rawmsf as a left join rawff as b
  on a.begdate<b.dateff and b.dateff<=a.enddate
  order by permno, date'''

msf=ps.sqldf(crsp_ff_join, locals())
msf['retrf']=msf['ret']-msf['rf']

In [25]:
# Import Compustat
rawccm=pd.read_stata(os.path.join(basepath, r"/content/ccm0820_short.dta"))
rawccm.columns=map(str.lower, rawccm.columns)
smallccm=rawccm[['fyear', 'gvkey', 'lpermno', 'at', 'sale', 'act', 'prcc_f',
'csho', 'seq', 'ceq', 'datadate', 'tic', 'conm']]
smallccm=smallccm.rename(columns={'lpermno':'permno'})
smallccm=smallccm[(smallccm['fyear']==2018)]
smallccm['beta_begdate']=smallccm['datadate']-pd.Timedelta(days=365*5)

# Form a 5-year rolling window
cond_join= '''select distinct a.gvkey, a.permno, a.fyear, a.tic, a.datadate,
b.retrf, b.ret, b.vwretd, b.rf, b.vwretd-b.rf as mktrf, b.date
  from smallccm as a left join msf as b
  on a.permno=b.permno and a.beta_begdate<b.date and b.date<=a.datadate+5
  order by a.permno, datadate, date'''
ccm_5year = ps.sqldf(cond_join, locals())

In [27]:
# Need to have available retrf and mktrf
ccm_5year=ccm_5year[(ccm_5year['retrf'].notnull()) &
(ccm_5year['mktrf'].notnull())]

def GroupRegress(dset, yvar, xvars):
  Y = dset[yvar]
  X = dset[xvars]
  X['intercept'] = 1.
  X = sm.add_constant(X)
  result = sm.OLS(Y, X).fit()
  stats= result.params
  return stats

In [28]:
capm = ccm_5year.groupby(['permno', 'fyear']).apply(GroupRegress, 'retrf',
['mktrf']).reset_index()
capm = capm.rename(columns={'intercept':'alpha', 'mktrf':'beta'})
ccm_beta=smallccm.merge(right=capm, how="left", on=["permno", 'fyear'])

In [29]:
# See Apple and MSFT
ticker_of_interest=['TGT', 'AAPL', 'WMT', 'MSFT']
for ticker in ticker_of_interest:
  beta=ccm_beta.loc[ccm_beta['tic']==ticker]['beta'].values
  print('{}\'s CAPM Beta is {}'.format(ticker, beta))

TGT's CAPM Beta is [0.83997323]
AAPL's CAPM Beta is [1.22845717]
WMT's CAPM Beta is [0.3140959]
MSFT's CAPM Beta is [0.78371831]


In [31]:
# See Keurig Dr Pepper
ticker_of_interest=['KDP']
for ticker in ticker_of_interest:
  beta=ccm_beta.loc[ccm_beta['tic']==ticker]['beta'].values
  print('{}\'s CAPM Beta is {}'.format(ticker, beta))

KDP's CAPM Beta is [0.5949923]
